In [1]:
import pandas as pd
import os
from openai import OpenAI

In [2]:
current_directory = os.getcwd()

parent_directory = os.path.dirname(current_directory)

DATA_PATH = os.path.join(parent_directory, "data")
DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")

DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")
DATA_BUSINESS = os.path.join(DATA_PATH, "Business category taxonomy.xlsx")
DATA_NAICS = os.path.join(
    DATA_PATH,
    "naics3_tournament_taxonomy.xlsx",
)

openai_api_key = "sk-proj-OSCk0tPAJxvpKtxmZIObT3BlbkFJA9hXFC8sq06DAxMmJgQl"

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

Current Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament/marian
Parent Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament


In [3]:
df_naics = pd.read_excel(DATA_NAICS)
df_business = pd.read_excel(DATA_BUSINESS)

In [4]:
list(df_naics["naics_label"])

['Crop Production',
 'Animal Production and Aquaculture',
 'Forestry and Logging',
 'Fishing, Hunting and Trapping',
 'Support Activities for Agriculture and Forestry',
 'Oil and Gas Extraction',
 'Mining (except Oil and Gas)',
 'Support Activities for Mining',
 'Utilities',
 'Construction of Buildings',
 'Heavy and Civil Engineering Construction',
 'Specialty Trade Contractors',
 'Food Manufacturing',
 'Beverage and Tobacco Product Manufacturing',
 'Textile Mills',
 'Textile Product Mills',
 'Apparel Manufacturing',
 'Leather and Allied Product Manufacturing',
 'Wood Product Manufacturing',
 'Paper Manufacturing',
 'Printing and Related Support Activities',
 'Petroleum and Coal Products Manufacturing',
 'Chemical Manufacturing',
 'Plastics and Rubber Products Manufacturing',
 'Nonmetallic Mineral Product Manufacturing',
 'Primary Metal Manufacturing',
 'Fabricated Metal Product Manufacturing',
 'Machinery Manufacturing',
 'Computer and Electronic Product Manufacturing',
 'Electrical E

In [5]:
client = OpenAI(api_key=openai_api_key)

In [6]:
from tqdm import tqdm

In [7]:
# def find_naics_category(label, description, naics_df):
#     responses = []

#     # print(f"Try to function {description} with {label}")

#     prompt = f"Given` the description: '{description}' with label {label}, choose one of the following naics taxonomy {df_naics["naics_label"]} "

#     try:
#         response = client.chat.completions.create(
#             model="gpt-4",
#             messages=[
#                 {"role": "user", "content": prompt},
#             ],
#             max_tokens=90,
#             temperature=0,
#         )

#         # responses.append((label, response.choices[0].message.content.strip()))

#     except Exception as e:
#         print(f"An error occurred: {e}")

#     # result = sorted(responses, key=lambda x: x[1], reverse=True)[0][0]

#     # print(f"{result} ----> {description}")

#     return result


# # Mapping each business label to a NAICS category
# df_business["Mapped NAICS Label"] = df_business["description"].apply(
#     lambda desc: find_naics_category(desc, df_naics)
# )

# # Saving the mapped labels
# df_business.to_csv("mapped_business_labels.csv", index=False)

In [8]:
def find_naics_category(description, naics_df):
    responses = []

    naics_labels = naics_df["naics_label"].tolist()
    prompt = f"Given the description: '{description}', choose one of the following NAICS taxonomy: {naics_labels}"

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": prompt},
            ],
            max_tokens=90,
            temperature=0,
        )
        # Append the response content after stripping unnecessary spaces
        responses.append(response.choices[0].message.content.strip())

    except Exception as e:
        print(f"An error occurred: {e}")
        responses.append(None)  # Append None in case of an error

    result = responses[0] if responses else None

    # print(f"{result} -----> {description}")
    return result

In [9]:
import pandas as pd
from tqdm import tqdm

# Assuming find_naics_category and df_naics are defined, as well as df_business

# Patch pandas apply with tqdm for visual progress bar
tqdm.pandas()

# Apply the function with progress tracking
df_business["Mapped NAICS Label"] = df_business["description"].progress_apply(
    lambda desc: find_naics_category(desc, df_naics)
)

  0%|          | 2/603 [00:01<08:26,  1.19it/s]

'Motor Vehicle and Parts Dealers' -----> Businesses categorized under ATVs Dealers & Services specialize in the sale, maintenance, and rental of all-terrain vehicles (ATVs). These establishments offer a range of services related to ATVs, including sales of new and used vehicles, repair and maintenance services, as well as rental options for recreational use. Customers can expect expert guidance on ATV selection, repair services to ensure optimal performance, and convenient rental options for outdoor adventures. The industry is dedicated to meeting the diverse needs of ATV enthusiasts, providing a one-stop destination for all ATV-related requirements.


  0%|          | 3/603 [00:02<08:19,  1.20it/s]

'Ambulatory Health Care Services' -----> Abortion Clinics provide medical services related to pregnancy termination in a clinical setting. These facilities offer a range of reproductive health services, including counseling, medical procedures, and follow-up care. Patients seeking abortion services may visit these clinics for confidential and professional healthcare assistance.


  1%|          | 4/603 [00:03<09:15,  1.08it/s]

...

'Chemical Manufacturing' -----> Adhesives & Sealants encompass a diverse range of products used in various industries for bonding, sealing, and coating applications. These materials play a crucial role in manufacturing processes by providing adhesion and protection to different surfaces. Adhesives are used to join materials together, while sealants are utilized to prevent leakage and provide a barrier against environmental factors. The category includes a wide array of specialized products tailored to meet specific industrial needs, ensuring reliable and durable bonding solutions.


  1%|          | 7/603 [00:06<08:35,  1.16it/s]

'Social Assistance' -----> Adoption Agencies & Foster Care involve organizations that facilitate the placement of children in need of permanent homes or temporary care. These entities play a crucial role in providing support and services to children, families, and caregivers involved in the adoption and foster care processes. They work to ensure the well-being and safety of children by overseeing the placement, monitoring, and support services throughout the adoption or foster care journey.


  1%|          | 7/603 [00:07<10:15,  1.03s/it]


KeyboardInterrupt: 

In [15]:
df_business["gpt4-label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

df_business.to_csv("mapped_business_labels_gpt.csv", index=False)

'Motor Vehicle and Parts Dealers'
'Ambulatory Health Care Services'
'Professional, Scientific, and Technical Services'
'Ambulatory Health Care Services'
'Chemical Manufacturing'
'Social Assistance'
'Motion Picture and Sound Recording Industries'
'Educational Services'
'Amusement, Gambling, and Recreation Industries'
'Personal and Laundry Services'
'Transportation Equipment Manufacturing'
'Support Activities for Agriculture and Forestry'
'Transportation Equipment Manufacturing'
'Air Transportation'
'Justice, Public Order, and Safety Activities'
'Ambulatory Health Care Services'
'Ambulatory Health Care Services'
'Primary Metal Manufacturing'
'Ambulatory Health Care Services'
'Support Activities for Agriculture and Forestry'
'Administration of Environmental Quality Programs'
'Social Assistance'
'Support Activities for Agriculture and Forestry'
'Miscellaneous Manufacturing'
'Publishing Industries'
'Repair and Maintenance'
'Furniture, Home Furnishings, Electronics, and Appliance Retailers'


In [ ]:
def find_naics_category_second_approach(labels, naics_df):
    responses = []

    # print(f"Try to function {description} with {label}")

    for label, label_description in zip(
        naics_df["naics_label"], naics_df["description"]
    ):
        prompt = f"Given` the description: '{label_description}' with  '{label}',choose one of the following labels: '{list(naics_df["nics"])}' which is described as: '{label_description}'. Compute and return a probability score from 0 to 1. Return just the probability score."

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    # {
                    #     "role": "system",
                    #     "content": "You are tasked with evaluating business descriptions and categorizing them based on a likelihood score. For each provided description and category, compute and return a probability score from 0 to 1.",
                    # },
                    {"role": "user", "content": prompt},
                ],
                max_tokens=90,
                temperature=0,
            )

            responses.append((label, response.choices[0].message.content.strip()))

        except Exception as e:
            print(f"An error occurred: {e}")

    print(responses)

    result = sorted(responses, key=lambda x: x[1], reverse=True)[0][0]

    return result


# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

In [23]:
test = df_business[:100]

In [24]:
test

,label,description
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...
1,Abortion Clinics,Abortion Clinics provide medical services rela...
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...
...,...,...
95,Butcher shop,A butcher shop is a retail establishment speci...
96,Buyback Shops,Buyback Shops are retail establishments that s...
97,CBD and Medical Marijuana,The CBD and Medical Marijuana category encompa...
98,Cabinet Makers,Cabinet Makers specialize in the skilled craft...
